#### Universidad del Valle de Guatemala<br>Data Science (20)<br>Grupo#5

#### Proyecto Final: Identificación de mosquitos

In [1]:
!pip install -U ultralytics tqdm opencv-python torchvision pandas Pillow gdown aicrowd-cli

  Using cached GitPython-3.1.18-py3-none-any.whl (170 kB)
  Attempting uninstall: GitPython
    Found existing installation: GitPython 3.1.40
    Uninstalling GitPython-3.1.40:
      Successfully uninstalled GitPython-3.1.40


In [2]:
import torch
import os
from pathlib import Path
import cv2
from timeit import default_timer as timer
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image

In [4]:
!aicrowd dataset download --challenge mosquitoalert-challenge-2023

phase2_train_v0.csv: 100% 553k/553k [00:00<00:00, 1.32MB/s]
phase2_train_v0.zip: 100% 10.3G/10.3G [19:51<00:00, 8.65MB/s]


In [13]:
!unzip -qq phase2_train_v0.zip -d test_images/

In [6]:
# Downloading the weight file at https://drive.google.com/file/d/1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS/view
!gdown "1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS"

Downloading...
From: https://drive.google.com/uc?id=1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS
To: /content/mosquitoalert-yolov5-baseline.pt
100% 14.4M/14.4M [00:00<00:00, 102MB/s] 


In [14]:
TEST_DATA_DIR = 'test_images/final'

In [8]:
def classify_image(image):
    image_information = {}
    result = yolov5_model(image)
    result_df = result.pandas().xyxy[0]
    if result_df.empty:
        print('No results from yolov5 model!')
    else:
        image_information = result_df.to_dict()
    return image_information

# getting mosquito_class name from predicted result
def extract_predicted_mosquito_class_name(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("name").get(0))
    return mosquito_class

# getting mosquito_class number from predicted result
def extract_predicted_mosquito_class_number(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("class").get(0))
    return mosquito_class

# getting mosquito_class confidence score from predicted result
def extract_predicted_mosquito_class_confidence(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("confidence").get(0))
    return mosquito_class

# getting mosquito bounding box from predicted result
def extract_predicted_mosquito_bbox(extractedInformation):
    bbox = []
    if extractedInformation is not None:
        xmin = str(extractedInformation.get("xmin").get(0))
        ymin = str(extractedInformation.get("ymin").get(0))
        xmax = str(extractedInformation.get("xmax").get(0))
        ymax = str(extractedInformation.get("ymax").get(0))
        bbox = [xmin, ymin, xmax, ymax]
    return bbox

# find image id
def find_image_id(original_image):
    image_name = os.path.splitext(original_image)[0]
    return image_name

In [15]:
# path to dataset
root_images = os.path.join(TEST_DATA_DIR)

all_images = os.listdir(root_images)
print(f"Total images: {len(all_images)}")

class_labels = {
    "aegypti":      0,
    "albopictus":   1,
    "anopheles":    2,
    "culex":        3,
    "culiseta":     4,
    "japonicus/koreicus": 5
}

# counter for correctly recognized mosquito classes
counter = 0
labels_counter = 0
no_mosquito_detected = 0
rows = []

# yolov5 challenge trained baseline
trained_model_pretrained = "mosquitoalert-yolov5-baseline.pt"

# yolov5_model
# Colab gets an error on this -> Use Runtime -> Restart Session (The data will not be deleted)
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path = trained_model_pretrained, force_reload = True)

for original_image in tqdm(all_images):
    try:
        original_image_file = os.path.join(root_images, original_image)
        # checking if it is a file
        if os.path.isfile(original_image_file):
            predictedInformation = classify_image(original_image_file)
            mosquito_class_name_predicted = ""
            mosquito_class_number_predicted = ""
            mosquito_class_confidence = 0
            mosquito_class_bbox = [0, 0, 0, 0]

            if predictedInformation:
                mosquito_class_name_predicted = extract_predicted_mosquito_class_name(predictedInformation)
                mosquito_class_number_predicted = extract_predicted_mosquito_class_number(predictedInformation)
                mosquito_class_confidence = extract_predicted_mosquito_class_confidence(predictedInformation)
                mosquito_class_bbox = extract_predicted_mosquito_bbox(predictedInformation)
                counter += 1
            else:
                no_mosquito_detected += 1
            #  bbox = [xmin, ymin, xmax, ymax]
            # create row for csv file
            row = [original_image, mosquito_class_name_predicted, mosquito_class_number_predicted, mosquito_class_confidence,
                   mosquito_class_bbox[0], mosquito_class_bbox[1], mosquito_class_bbox[2], mosquito_class_bbox[3]]
            rows.append(row)
            labels_counter += 1
            # print(f"Finished file reading, file {original_image} read correctly!")
    except Exception as e:
        print(f'Unable to process file: {original_image}!')
        print(f'Exception: {e}!')

Total images: 10357


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-10-23 Python-3.10.12 torch-2.1.0+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


  0%|          | 0/10357 [00:00<?, ?it/s]

No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3182: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!


In [16]:
df = pd.DataFrame(rows, columns=["img_fName", "class_label", "predicted_class_number", "confidence_score", "bbx_xtl", "bbx_ytl", "bbx_xbr", "bbx_ybr"])
sub_df = df.drop(['predicted_class_number', 'confidence_score'], axis=1, inplace=False)
sub_df.head()

,img_fName,class_label,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr
0,train_01944.jpeg,culex,1309.7489013671875,952.150146484375,2114.6962890625,1771.1876220703125
1,train_08491.jpeg,albopictus,1742.198974609375,673.3878784179688,1963.29638671875,930.0755004882812
2,train_08396.jpeg,culex,624.63427734375,936.7860717773438,1361.159912109375,1620.7274169921875
3,train_04082.jpeg,culex,32.13994216918945,256.6070251464844,498.9612731933594,623.1806640625
4,train_05915.jpeg,albopictus,584.4896240234375,1625.939453125,1353.993408203125,2243.467529296875


In [18]:
sub_df.to_csv('Results.csv', index=False)